In [55]:
import spacy
import classy_classification
import glob
import pandas as pd

In [49]:
label_files = glob.glob('labels/*.txt')

labels = {}

for lf in label_files:
    label = lf.split('/')[1].split('.')[0]
    examples = [e.strip() for e in open(lf).readlines()]
    labels[label] = examples

print(labels)


{'body': ['My head aches, and I will need some rest.', 'I feel a sense of hunger in my stomach.', 'She was unable to straighten her fingers.'], 'clothes': ['I will get my hat and coat, and I think also my umbrella.', 'Shoes are good to wear.', 'The trousers are striped, and my pockets are empty.'], 'ghosts': ['This is spooky, and the night is dark.', 'A phantom is fleeting across my room.', 'A ghastly shape is hiding behind my curtain.']}


In [50]:
# start with a blank spacy model
nlp = spacy.blank("en") 

# add in the text_categorizer from classy_classification to the processing pipeline
nlp.add_pipe("text_categorizer",
    config ={
    "data": labels,
    "model": "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",
    "device": "cpu"
    }
)


In [53]:
sentences = [i.strip() for i in open('data/unseen.txt').readlines()]

In [63]:
# run classification
sents = []
scores = []


for s in sentences:
    sents.append(s)
    scores.append(nlp(s)._.cats)
    break

df = pd.DataFrame(zip(sents,scores), columns = ['text', 'scores'])
df = pd.concat([df.drop(['scores'], axis=1), df['scores'].apply(pd.Series)], axis=1)
df.to_csv('bambambam.csv', index = False)

In [61]:
import pandas as pd

# create example DataFrame
df = pd.DataFrame({'dict_col': [{'key1': 'value1', 'key2': 'value2'},
                                {'key1': 'value3', 'key2': 'value4'}]})

# apply pd.Series constructor to each row of dict_col
df = pd.concat([df.drop(['dict_col'], axis=1), df['dict_col'].apply(pd.Series)], axis=1)
df

,key1,key2
0,value1,value2
1,value3,value4


In [39]:
# run classification
print(nlp("I am looking for kitchen appliances.")._.cats)

{'furniture': 0.12022529596456778, 'kitchen': 0.8797747040354322}


In [26]:
import spacy
nlp = spacy.load('en_core_web_sm')

text = 'My first birthday was great. My 2. was even better.'
sentences = [i for i in nlp(text).sents]


In [27]:
sentences

[My first birthday was great., My 2. was even better.]

In [17]:
sentence_model = spacy.blank("en")

sentence_model.add_pipe("sentencizer")

with open ("harry_potter_cleaned.txt", "r") as f:
    text = f.read()


sentences = sentence_model(text)

In [20]:
sentences[4]

,

In [29]:
final_data = []
for sentence in sentences.sents:
    doc = nlp(sentence.text) #< -- perform the categorizer at the sentence level
    final_data.append({"sentence": doc.text, "cats": doc._.cats})

In [30]:
final_data[0]

{'sentence': 'Mr. and Mrs. Dursley, of number four, Privet Drive, were proud to say that they were perfectly normal, thank you very much.',
 'cats': {'furniture': 0.588502517277626, 'kitchen': 0.4114974827223741}}

In [31]:
for item in final_data:
    if item["cats"]["fear"] > .8:
        print (item["sentence"].strip())
        print (item["cats"])
        print ()

KeyError: 'fear'